# Setup

**Step 1**: Import Semantic Kernel SDK from pypi.org

In [1]:
!python -m pip install semantic-kernel==0.2.7.dev0

**Step 2**: Import the relevant packages required to run the notebook

In [2]:
from typing import Tuple

import json
import csv

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion, OpenAITextEmbedding

**Step 3**: Run the below cell to create a json file which is required to be fed into the memory of the semantic kernel. This step requires a csv file which can be dowloaded from the website https://www.dol.gov/agencies/ebsa/about-ebsa/our-activities/public-disclosure/foia/form-5500-datasets. 

A sample csv file 'f_5500_sf_2022_latest.csv' has been dowloaded and placed in the same folder as the notebook. A column mapping for the csv file has also been provided below which converts short forms to full forms. This helps the LLM better understand the columns and generate optimal answers

**NOTE:-** 
1. Run the below cell to create a json file.
2. Not required to be run if the json file has already been created.
3. Number of rows to extract from the csv can be controlled using the **num_rows_to_extract** variable.

In [3]:
num_rows_to_extract = 100

def csv_to_json(csv_file_path, columns, num_rows):
    json_data = {}
    row_count = 0

    
    with open(csv_file_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            if row_count >= num_rows:
                break
            
            json_row = {}
            for column in columns:
                full_column_name = column_mapping.get(column, column)
                json_row[full_column_name] = row[column]
            
            key = 'row{}'.format(row_count + 1)
            json_data[key] = str(json_row)
            
            row_count += 1
    
    return json.dumps(json_data, indent=4)

# Define a mapping of column names from CSV to their full forms
column_mapping = {
    'ACK_ID': 'AcknowledgementId',
    'SF_PLAN_YEAR_BEGIN_DATE': 'PlanYearBeginDate',
    'SF_PLAN_NAME': 'PlanName',
    'SF_SPONSOR_NAME': 'SponsorName',
    'SF_SPONS_US_ADDRESS1': 'SponsorUsAddress1',
    'SF_SPONS_US_CITY': 'SponsorUsCity',
    'SF_SPONS_US_STATE': 'SponsorUsState',
    'SF_SPONS_EIN': 'SponsorEin',
    'SF_SPONS_PHONE_NUM': 'SponsorPhoneNumber',
    'SF_BUSINESS_CODE': 'BusinessCode',
    'SF_TOT_ASSETS_BOY_AMT': 'TotalAssets',
    'SF_TOT_EXPENSES_AMT': 'TotalExpenses',
    'SF_ADMIN_SIGNED_NAME': 'SignedAdminName'
}

csv_file_path = 'f_5500_sf_2022_latest.csv'
columns_to_extract = column_mapping.keys()

json_data = csv_to_json(csv_file_path, columns_to_extract, num_rows_to_extract)

with open('output.json', 'w') as jsonfile:
    jsonfile.write(json_data)


**Step 4**: Add your [Open AI Key](https://openai.com/api/) key to a `.env` file in the same folder (org Id only if you have multiple orgs):

```
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
```

add OpenAI Text Completion and OpenAI Embedding model to the kernel.

This initializes and instance of the semantic kernel:-

In [4]:
kernel = sk.Kernel()

api_key, org_id = sk.openai_settings_from_dot_env()
kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))
kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x2397f668490>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x2397f668730>}

**Step 5**: Importing and reading the JSON file which will be fed into memory

In [5]:
with open("output.json","r") as f:
    memory_context_dictionary = json.load(f)
# print(memory_context_dictionary)

**Step 6**: 

In order to use memory, we need to instantiate the Kernel with a Memory Storage
and an Embedding service. In this example, we make use of the `VolatileMemoryStore` "which can be thought of as a temporary in-memory storage (not to be confused with Semantic Memory). This memory is not written to disk and is only available during the app session.

We can add memories to our `VolatileMemoryStore` by using `save_reference_async`

The texts are embedded or compressed into a vector of floats representing mathematically the texts' contents and meaning. 

In [6]:
memory_collection_name = "FormData"
print("Adding form data to semantic kernel dictionary")
i = 0
for entry, value in memory_context_dictionary.items():
    await kernel.memory.save_reference_async(
        collection=memory_collection_name,
        description=value,
        text=value,
        external_id=entry,
        external_source_name="Form5500"
    )
    i += 1
    print("  Data {}/{} saved".format(i,len(memory_context_dictionary)),end="\r")

Adding form data to semantic kernel dictionary


**Step 7**:

Once the data is stored in memory, the kernel memory is queried using search_async function and user prompt as input to fetch all the top relevant chunks to the user prompt. The relevant chunks to be fetched can be controlled using the 'limit' parameter. The chunks that have relevance score greater than min_relevance_score parameter value are fetched.

Once the relevant chuncks are extracted from memory, the chunks are clubbed together and combined with the user prompt to act as context. The combined string is then passed to the semantic kernel to generate a response.

In [7]:
async def prompt_answer(prompt):

    # query to extract relevant chunck from memory
    memories = await kernel.memory.search_async(memory_collection_name, prompt, limit=5, min_relevance_score=0.7)

    memoryFeed = "I dont know"

    # relevant memory chunks are clubbed together
    for memory in memories:
        memoryFeed = memoryFeed + '\n' + memory.description
        # print(memory.description)
        # print(str(memory.relevance))


    # relevant memory chunks are combined with the user prompt
    sk_prompt = """

    {{$input}}

    """+'\n'+prompt

    chat_function = kernel.create_semantic_function(sk_prompt, "policyBot", max_tokens=2000, temperature=0.7, top_p=0.5)

    bot_answer = await chat_function.invoke_async(memoryFeed)
    print("Q: "+prompt)
    print("Answer: "+str(bot_answer))
    print('\n')

Below are a set of sample questions. The function prompt_answer is called to generate an answer for the prompt

In [8]:
ask_list = [
"which plans began in the year 2022?",
 "what plans have assests allocated in the range 900000 and 1000000?",
 "which 401k plans are based out of New York State?",
 "Which all organizations offer 401k plans, which are not based out of New york State and which have assets greater than 1000000?",
]

for ask in ask_list:
    await prompt_answer(ask)

Q: which plans began in the year 2022?
Answer: 

The plans that began in the year 2022 are:
- FORM TOOL TECHNOLOGY, INC. 401(K) PROFIT SHARING PLAN
- ADVANCED PLASTIC SURGERY 401(K) PLAN
- ADVANCED AESTHETIC SURGERY, PC 401(K) PLAN
- AXIS DIAGNOSTICS, INC. 401(K) PLAN
- BAYSIDE CONCRETE 401(K) PLAN


Q: what plans have assests allocated in the range 900000 and 1000000?
Answer: 

The plans with assets allocated in the range 900000 and 1000000 are:

- Bayside Concrete 401(k) Plan 
- Axis Diagnostics, Inc. 401(k) Plan


Q: which 401k plans are based out of New York State?
Answer: 

The following 401k plans are based out of New York State:

- SFG Wealth Planning Services, Inc. 401(K) Plan
- Bayside Concrete 401(K) Plan
- Office IT Solutions 401K Retirement Plan
- Art Craft Display Inc 401K Retirement Plan
- Kathleen Edmunds, MD 401K Profit Sharing Plan


Q: Which all organizations offer 401k plans, which are not based out of New york State and which have assets greater than 1000000?
Answer

Use the below cell to pass any prompt to the semantic kernel through the 'user_prompt' variable

In [9]:
user_prompt = "Name a few companies that have benefit plans whose allocated funds are greater than 10000"

await prompt_answer(user_prompt)

Q: Name a few companies that have benefit plans whose allocated funds are greater than 10000
Answer:  USD

1. SFG Wealth Planning Services, Inc.
2. RR Company of America, LLC
3. Advanced Plastic Surgery Institute, PLLC
4. Kathleen Edmunds, MD
5. Form Tool Technology, Inc.


